# Quick Plot of Variance of Consumption Growth vs Market Resources

This notebook makes a quick plot of the variance of consumption growth $Var[\Delta \log C_{t+1}]$ vs normalized market resources $m_t$.  It was prompted by a comment by CDC to MNW re: a prior paper by Dimitris Georgarakos.

In [ ]:
# This cell has just a bit of initial setup. You can click the arrow to expand it.
%matplotlib inline
import matplotlib.pyplot as plt

# The first step is to be able to bring things in from different directories
import sys 
import os
sys.path.insert(0, os.path.abspath('../lib'))

import numpy as np
import HARK # Prevents import error from Demos repo
from time import clock
from copy import deepcopy
mystr = lambda number : "{:.4f}".format(number)
from HARK.utilities import plotFuncs

In [ ]:
# This cell defines a parameter dictionary for making an instance of IndShockConsumerType.

IndShockDictionary = {
    'CRRA': 2.5,          # The dictionary includes our original parameters...
    'Rfree': 1.03,
    'DiscFac': 0.96,
    'LivPrb': [0.98],
    'PermGroFac': [1.01],
    'PermShkStd': [0.1], 
    'PermShkCount': 7,
    'TranShkStd': [0.1],
    'TranShkCount': 7,
    'UnempPrb': 0.05,
    'IncUnemp': 0.3,
    'BoroCnstArt': 0.0,
    'aXtraMin': 0.001,  
    'aXtraMax': 50.,      
    'aXtraNestFac': 3,
    'aXtraCount': 48,
    'aXtraExtra': [None],
    'vFuncBool': False,   
    'CubicBool': False,   
    'aNrmInitMean' : -10.,
    'aNrmInitStd' : 0.0,  
    'pLvlInitMean' : 0.0,
    'pLvlInitStd' : 0.0,  
    'PermGroFacAgg' : 1.0,
    'T_retire': 0,        
    'UnempPrbRet': 0.0,   
    'IncUnempRet': 0.0,   
    'T_age' : None,
    'T_cycle' : 1,
    'cycles' : 0,
    'AgentCount': 10000,
    'tax_rate':0.0,
}

In [ ]:
# Create some agents, solves them, and extracts the consumption function
from Demos.ConsumptionSaving.ConsIndShockModel  import IndShockConsumerType
DimitrisType = IndShockConsumerType(**IndShockDictionary)
DimitrisType.solve()
cFunc = DimitrisType.solution[0].cFunc

In [ ]:
# Draw income shocks and reshape them into tiled arrays
N = DimitrisType.AgentCount
M = 200
DimitrisType.T_sim = 1
DimitrisType.initializeSim()
DimitrisType.makeShockHistory()
PermShkArray = np.tile(np.reshape(DimitrisType.PermShkNow,(1,N)),(M,1))
TranShkArray = np.tile(np.reshape(DimitrisType.TranShkNow,(1,N)),(M,1))

In [ ]:
# Define a grid of current market resources and reshape it into tiled form
mNrmNowGrid = np.linspace(0.1,10.,M)
mNrmNowArray = np.tile(np.reshape(mNrmNowGrid,(M,1)),(1,N))

# Do the same thing for consumption today
cNrmNowGrid = cFunc(mNrmNowGrid)
cNrmNowArray = np.tile(np.reshape(cNrmNowGrid,(M,1)),(1,N))
cLvlNowArray = 1.0*cNrmNowArray

In [ ]:
# Calculate future market resources and future consumption
aNrmNowArray  = mNrmNowArray - cNrmNowArray
mNrmNextArray = DimitrisType.Rfree*aNrmNowArray/PermShkArray + TranShkArray
cNrmNextArray = cFunc(mNrmNextArray)
cLvlNextArray = PermShkArray*cNrmNextArray 

In [ ]:
# Calculate log consumption growth
cGroArray = np.log(cLvlNextArray/cLvlNowArray)

# Calculate the mean and variance of consumption growth by mNrmNow
cGroMean = np.mean(cGroArray,axis=1)
cGroVar  = np.var(cGroArray,axis=1)

In [ ]:
# Make linear interpolation functions of mean and variance of consumption growth
from HARK.interpolation import LinearInterp
cGroMeanFunc = LinearInterp(mNrmNowGrid,cGroMean)
cGroVarFunc = LinearInterp(mNrmNowGrid,cGroVar)

In [ ]:
# Plot the consumption function, mean consumption growth, and variance consumption growth functions
plt.xlabel('Market resources')
plt.ylabel('Consumption')
plotFuncs(cFunc,0.,10.)

plt.xlabel('Market resources')
plt.ylabel('Mean consumption growth')
plotFuncs(cGroMeanFunc,0.,10.)

plt.xlabel('Market resources')
plt.ylabel('Variance consumption growth')
plotFuncs(cGroVarFunc,0.,10.)